In [96]:
import pandas as pd
import numpy as np
import requests

#taking data from ergast via json and also wherever data is not there, we are putting None there

#Making dictionary of lists for needful features
races = {'season': [],
        'round': [],
        'circuit_id': [],
        'lat': [],
        'long': [],
        'country': [],
        'date': [],
        'url': []}

#For now we are taking data from 2015 to 2022 season
for year in list(range(2015,2023)):
    
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            races['season'].append(int(item['season']))
        except:
            races['season'].append(None)

        try:
            races['round'].append(int(item['round']))
        except:
            races['round'].append(None)

        try:
            races['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            races['circuit_id'].append(None)

        try:
            races['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            races['lat'].append(None)

        try:
            races['long'].append(float(item['Circuit']['Location']['long']))
        except:
            races['long'].append(None)

        try:
            races['country'].append(item['Circuit']['Location']['country'])
        except:
            races['country'].append(None)

        try:
            races['date'].append(item['date'])
        except:
            races['date'].append(None)

        try:
            races['url'].append(item['url'])
        except:
            races['url'].append(None)
        
races = pd.DataFrame(races)

In [97]:
#creating an excel of races for data analysis
races.to_csv("Races2015_2022.csv")

In [98]:
#iterating through each season to get driver data and winner's data for each race ex. podium data
import numpy as np

#Making rounds list so that we can iterate later on through it for data fetching for each round
rounds = []
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])

#Making dictionary of lists for needful features

results = {'season': [],
          'round':[],
           'circuit_id':[],
          'driver': [],
           'date_of_birth': [],
           'nationality': [],
          'constructor': [],
          'grid': [],
          'time': [],
          'status': [],
          'points': [],
          'podium': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:
            try:
                results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            except:
                results['season'].append(None)

            try:
                results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            except:
                results['round'].append(None)

            try:
                results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            except:
                results['circuit_id'].append(None)

            try:
                results['driver'].append(item['Driver']['driverId'])
            except:
                results['driver'].append(None)
            
            try:
                results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            except:
                results['date_of_birth'].append(None)
                
            try:
                results['nationality'].append(item['Driver']['nationality'])
            except:
                results['nationality'].append(None)

            try:
                results['constructor'].append(item['Constructor']['constructorId'])
            except:
                results['constructor'].append(None)

            try:
                results['grid'].append(int(item['grid']))
            except:
                results['grid'].append(None)

            try:
                results['time'].append(int(item['Time']['millis']))
            except:
                results['time'].append(None)

            try:
                results['status'].append(item['status'])
            except:
                results['status'].append(None)

            try:
                results['points'].append(int(item['points']))
            except:
                results['points'].append(None)

            try:
                results['podium'].append(int(item['position']))
            except:
                results['podium'].append(None)

           
results = pd.DataFrame(results)

In [99]:
#creating an excel of results for data analysis
results.to_csv("Results2015_2022.csv")

In [100]:
#Making dictionary of lists for needful features
driver_standings = {'season': [],
                    'round':[],
                    'driver': [],
                    'driver_points': [],
                    'driver_wins': [],
                   'driver_standings_pos': []}

#Making driver standings list for each round
for n in list(range(len(rounds))):     
    for i in rounds[n][1]:    # iterate through rounds of each year
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            try:
                driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                driver_standings['season'].append(None)

            try:
                driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                driver_standings['round'].append(None)
                                         
            try:
                driver_standings['driver'].append(item['Driver']['driverId'])
            except:
                driver_standings['driver'].append(None)
            
            try:
                driver_standings['driver_points'].append(int(item['points']))
            except:
                driver_standings['driver_points'].append(None)
            
            try:
                driver_standings['driver_wins'].append(int(item['wins']))
            except:
                driver_standings['driver_wins'].append(None)
                
            try:
                driver_standings['driver_standings_pos'].append(int(item['position']))
            except:
                driver_standings['driver_standings_pos'].append(None)
            
driver_standings = pd.DataFrame(driver_standings)

In [101]:
#need to create a function which will add up points as points are given after race completion so they needs to be added up in the next race 
#to shift points and number of wins from previous rounds
def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df
  
driver_standings = lookup(driver_standings, 'driver', 'driver_points')
driver_standings = lookup(driver_standings, 'driver', 'driver_wins')
driver_standings = lookup(driver_standings, 'driver', 'driver_standings_pos')

driver_standings.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'], 
                      axis = 1, inplace = True)

In [102]:
#creating an excel of driver standings for data analysis
driver_standings.to_csv("Dr_standings2015_2022.csv")

In [103]:
#Making dictionary of lists for needful features
constructor_rounds = rounds[:]
constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []}

#Making constructor standings list for each round
for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            try:
                constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                constructor_standings['season'].append(None)

            try:
                constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                constructor_standings['round'].append(None)
                                         
            try:
                constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            except:
                constructor_standings['constructor'].append(None)
            
            try:
                constructor_standings['constructor_points'].append(int(item['points']))
            except:
                constructor_standings['constructor_points'].append(None)
            
            try:
                constructor_standings['constructor_wins'].append(int(item['wins']))
            except:
                constructor_standings['constructor_wins'].append(None)
                
            try:
                constructor_standings['constructor_standings_pos'].append(int(item['position']))
            except:
                constructor_standings['constructor_standings_pos'].append(None)
            
constructor_standings = pd.DataFrame(constructor_standings)

In [104]:
#applying same lookup and logic here as driver stadings
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_points')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_wins')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_standings_pos')

constructor_standings.drop(['constructor_points_after_race', 'constructor_wins_after_race','constructor_standings_pos_after_race' ],
                           axis = 1, inplace = True)

In [105]:
#creating an excel of driver standings for data analysis
constructor_standings.to_csv("construct_standings2015_2022.csv")

In [106]:
#gathering qualifying results from 2015 to 2022 using beautiful soup via web scrapping
import bs4
from bs4 import BeautifulSoup

#creating empty dataframe
qualifying_results = pd.DataFrame()

for year in list(range(2015,2023)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')

    year_links = []
    #finding all circuits links for each year
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    #creating an empty dataframe
    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        #or each circuit, switching to the starting grid page and reading table from it
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)
        year_df = pd.concat([year_df, df])

    # concatenating all tables from all years now
    qualifying_results = pd.concat([qualifying_results, year_df])

#renaming columns for better understanding
qualifying_results.rename(columns = {'Pos': 'grid', 'Driver': 'driver_name', 'Car': 'car',
                                     'Time': 'qualifying_time'}, inplace = True)

#we don't need driver number here as we have driver name to identify so dropping it
qualifying_results.drop('No', axis = 1, inplace = True)

In [107]:
#creating an excel of qualifying results
qualifying_results.to_csv("Qualifying2015_2022.csv")

In [108]:

#dropping unnecessary columns from races dataframe before merging
races.drop(['lat', 'long','country'], axis=1, inplace=True)

In [109]:
#merging all dataframes to create one final dataframe
df1= pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], axis = 1)

In [110]:
df2= pd.merge(df1, driver_standings, how='left', on=['season', 'round', 'driver'])

In [111]:
df3 = pd.merge(df2, constructor_standings, how='left', on=['season', 'round', 'constructor'])

In [112]:
final_df = pd.merge(df3, qualifying_results, how='inner', on=['season', 'round', 'grid']).drop(['driver_name', 'car'], axis = 1)

In [113]:
# calculating age of drivers as we will take it as a parameter as it impact health due to g torque.
from dateutil.relativedelta import *

final_df['date'] = pd.to_datetime(final_df.date)
final_df['date_of_birth'] = pd.to_datetime(final_df.date_of_birth)
final_df['driver_age'] = final_df.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)
final_df.drop(['date', 'date_of_birth'], axis = 1, inplace = True)

In [114]:
#Handling missing values
for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 'constructor_wins' , 'constructor_standings_pos']:
    final_df[col].fillna(0, inplace = True)
    final_df[col] = final_df[col].map(lambda x: int(x))

In [115]:
final_df.dropna(inplace = True )

In [116]:
#calculating difference of qualifying times so we can get how fast one car is from another

def convert_time_to_seconds(time_str):
    try:
        if time_str == '00.000':
            return 0
        else:
            if isinstance(time_str, float):
                print(time_str)
                print(type(time_str))
                return time_str   # Return the value as it is (already converted to float)
            else:
                minutes, seconds = map(float, time_str.split(':'))
                print("minute: ", minutes,"seconds: ", seconds)
                return minutes * 60 + seconds
    except (ValueError, IndexError):
        return 0
    
final_df['qualifying_time'] = final_df['qualifying_time'].map(convert_time_to_seconds)
final_df = final_df[final_df['qualifying_time'] != 0]
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round'])['qualifying_time'].diff()
final_df['qualifying_time'] = final_df.groupby(['season', 'round'])['qualifying_time_diff'].cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)

minute:  1.0 seconds:  26.327
minute:  1.0 seconds:  26.921
minute:  1.0 seconds:  27.757
minute:  1.0 seconds:  27.718
minute:  1.0 seconds:  28.8
minute:  1.0 seconds:  28.329
minute:  1.0 seconds:  29.208
minute:  1.0 seconds:  31.376
minute:  1.0 seconds:  28.51
minute:  1.0 seconds:  29.209
minute:  1.0 seconds:  31.422
minute:  1.0 seconds:  27.79
minute:  1.0 seconds:  28.868
minute:  1.0 seconds:  28.56
minute:  1.0 seconds:  29.48
minute:  1.0 seconds:  29.07
minute:  1.0 seconds:  32.037
minute:  1.0 seconds:  49.908
minute:  1.0 seconds:  49.834
minute:  1.0 seconds:  50.299
minute:  1.0 seconds:  42.173
minute:  1.0 seconds:  53.179
minute:  1.0 seconds:  52.473
minute:  1.0 seconds:  51.981
minute:  1.0 seconds:  43.701
minute:  1.0 seconds:  51.951
minute:  1.0 seconds:  51.541
minute:  1.0 seconds:  52.981
minute:  1.0 seconds:  41.308
minute:  1.0 seconds:  43.469
minute:  1.0 seconds:  43.023
minute:  1.0 seconds:  46.677
minute:  1.0 seconds:  42.198
minute:  1.0 seco

In [117]:
#Final preprocessed Excel
final_df.to_csv("Final2015_2022.csv")